In [ ]:
import sys

!{sys.executable} -m pip install yfinance --upgrade --no-cache-dir

# ML 응용 : 주식시장 데이터 클러스터링

**목표**: 고차원 공간에서 근접한 이웃을 추정해 냅니다.

1. 마켓플레이스의 [K-Shape: Time Series Clustering](https://aws.amazon.com/marketplace/pp/Spotad-LTD-K-Shape-Time-Series-Clustering/prodview-bjbovimwn5ajs) 알고리즘을 사용하여 시계열 데이터를 클러스터링 합니다.

1. SageMaker의 빌트인 [K-Means Algorithm](https://docs.aws.amazon.com/sagemaker/latest/dg/k-means.html) 알고리즘을 이용하여 고차원 데이터를 클러스터링합니다.



#### Task 1 설명:
첫 번째 과제에서는 시계열 데이터를 클러스터링하고, 주어진 기간 동안 서로 동일한 성과를 내는 주식을 찾아내는 방법을 배웁니다. 진행과정에서 주식시장 데이터를 다운로드 하고, 각 주가를 정규화한 후, 고유한 모양을 가지는 주식 클러스터를 식별할 것입니다. 그 다음 어떤 주식이 동일한 패턴을 보이는지 살펴보고 정리합니다. 여러가지 K (2~20)값을 시도하여 최소 SSD (Sum of the squared distances, 각 데이터포인트와 클러스터 중심사이의 거리 제곱의 합)를 리턴하는  `k`를 찾아냅니다. 

Task 2를 위해 보다 많은 시간을 투자하실 수 있도로 Task1은 보다 많은 구분을 이미 구현한 형태로 준비하였습니다.


#### *References:*

* [K-means 블로그](https://aws.amazon.com/blogs/machine-learning/k-means-clustering-with-amazon-sagemaker/)
* [AWS 마켓플레이스의 알고리즘과 모델을 이용하여 ML 프로젝트 가속화하기(동영상))](https://youtu.be/OrmHHVI1uPk?t=1682)
* [그래프 등 참고 예제](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/aws_marketplace/using_model_packages/financial_transaction_processing/Extracting_insights_from_your_credit_card_statement.ipynb)

#### Task 2 설명:
본 과제에서는 고차원 공간에서 클러스터링 알고리즘을 적용하여 어떻게 근접한 이웃을 근사하는지를 배우게 됩니다. 본 작업의 일부분으로, 처음은 주식 종목 포트폴리오를 포함하는 고차원 데이터셋을 합성할 것이고, 그 다음 이 데이터셋에 K-Means 클러스터링 알고리즘을 적용하여 유사한 포트폴리오를 가지는 투자자의 군집을 구별하게 됩니다. 

**Notes**:

* 프로젝트에 재미를 더하기 위해, 특정 종목을 리스트에 추가해 보십시오. 

* 프로젝트 시간이 남는 경우 Task1과 Task2 문제를 풀 수 있는 다른 알고리즘을 시도해 보고 적절한 매트릭으로 결과를 비교해 보십시오.

#### *References:*

* [K-means 블로그](https://aws.amazon.com/blogs/machine-learning/k-means-clustering-with-amazon-sagemaker/)
* [K-Means 동작방식(SageMaker 개발자 가이드)](https://docs.aws.amazon.com/sagemaker/latest/dg/algo-kmeans-tech-notes.html)


In [ ]:
# 본 실험에서는 다음 주식 종목 들을 사용하겠습니다.
tickers = ['FB','AAPL','MSFT','GOOGL','GOOG','JNJ','V','PG','JPM','UNH','HD','MA','INTC','NVDA','VZ','NFLX','ADBE','DIS','T','PYPL','PFE','MRK','CSCO','CMCSA','WMT','PEP','BAC','XOM','KO','CRM','ABBV','ABT','CVX','TMO','AMGN','COST','MCD','ACN','LLY','BMY','NEE','MDT','AVGO','LIN','TXN','DHR','UNP','NKE','AMT','ORCL','PM','IBM','LOW','HON','QCOM','C','GILD','BA','WFC','RTX','LMT','MMM','BLK','SBUX','FIS','SPGI','NOW','CHTR','CVS','UPS','VRTX','BDX','INTU','ISRG','MDLZ','MO','CAT','CCI','BKNG','PLD','ZTS','AMD','REGN','GS','ANTM','D','CI','EQIX','APD','ADP','CL','ATVI','MS','AXP','TJX','SYK','CB','TMUS','TGT']

In [ ]:
import os
import json
import boto3
import sagemaker
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import boto3
import sagemaker as sage
import yfinance as yf
import botocore
from sklearn import preprocessing
from uuid import uuid4
from collections import namedtuple
from functools import partial
from scipy.stats import zscore
from sagemaker import AlgorithmEstimator
from matplotlib.pyplot import figure
import warnings
import matplotlib.dates as mdates
from sagemaker.predictor import csv_serializer
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner
from sklearn.model_selection import train_test_split

warnings.filterwarnings("ignore")
%matplotlib inline

#visualization variables
palette=sns.color_palette("RdBu", n_colors=7)

In [ ]:
#Common variable declaration
region_name = boto3.Session().region_name
bucket=sage.Session().default_bucket()
role = sage.get_execution_role()
sagemaker_session = sage.Session()

환경에 따라 다음 warning이 발생할 수 있습니다. 셀이 성공적으로 실행되었는지 확인합니다. 

`Couldn't call 'get_role' to get Role ARN from role name Sagemaker_Studio_Role to get Role path.`

## Task 1:
본 프로젝트에서 여러분은 고유한 모양을 가지는 주식을 찾을 것입니다. 이 작업은 다음 세 가지 단계로 나누어집니다.

##### Step 1:
* 95일 동안의 주식시장 데이터를 다운로드하고 알고리즘에서 사용하는 포맷의 파일로 변환합니다. 알고리즘은 csv포맷의 정규화된 시계열 데이터를 요구합니다. 데이터의 각 행(row)은 특정 주식에 대한 시계열 정보로 표현되어야 합니다.

##### Step 2:
* 모양기반으로 시계열을 클러스터링하고 유사한 패턴의 주식 클러스터를 식별합니다. - 여기서 시계열의 크기(magnitude)가 아닌 모양(shape)에 집중하는 것에 주목해 주십시오. 

##### Step 3:
실험을 진행하고 발견한 내용을 보고합니다. 


첫번째 과제에서, [AWS Marketplace](https://aws.amazon.com/marketplace/search/results?page=1&filters=fulfillment_options&fulfillment_options=SAGEMAKER&ref_=header_nav_dm_sagemaker)에 있는 [K-Shape Time Series Clustering](https://aws.amazon.com/marketplace/pp/Spotad-LTD-K-Shape-Time-Series-Clustering/prodview-bjbovimwn5ajs) 알고리즘을 사용합니다. K-Shape Time Series Clustering 알고리즘은 다음 [연구 논문](
http://web2.cs.columbia.edu/~gravano/Papers/2015/sigmod2015.pdf)에 기반합니다.

In [ ]:
# 데이터를 다운로드할 대상 기간을 설정합니다. 
start_date = '2020-02-03'
end_date = '2020-06-18'
common_prefix = "k-shape-clustering"


In [ ]:
# 정의된 모든 종목에 대하여 주식 정보를 다운로드하고 데이터프레임을 생성합니다. 
data = yf.download(' '.join(tickers), start=start_date, end=end_date, group_by="ticker")

In [ ]:
data.head(10)

In [ ]:
# 주식값이 가용한 날짜를 뽑아 컬럼으로 지정합니다.
dates=data[tickers[0]]['Close'].index

In [ ]:
len(dates)
##df[dates].values

본 실험에서는 여러 측정값 중 종가(closing price)를 사용하겠습니다.

In [ ]:
close_data=[]

for ticker in tickers:
    ticker_data=[ticker]
    ticker_data.extend(data[ticker]['Close'].values)
    close_data.append(ticker_data)

#print('Closing price data set for ',len(close_data),' tickers')
#print(close_data[0])

리스트로 선언된 `close_data`는 종목별 주가의 시계열 정보를 담고 있습니다. 이 데이터를 데이터프레임으로 변환합니다.

In [ ]:
columns=['Ticker']
columns.extend(dates)

df=pd.DataFrame(data=close_data,columns=columns)

In [ ]:
df.head()

여기까지 다른 문제는 없어보입니다. 이제 이 데이터를 행(row) 단위로 정규화하고 파일로 저장합니다. 

In [ ]:
x = df[dates].values
len(x)

In [ ]:
x[0]

In [ ]:
# 컬럼(column)이 아닌 행(row)을 기준으로 정규화하기 위해 먼저 전치(transpose)한 후 MinMaxScaler를 적용합니다. .
# 정규화 이후에는 다시 결과를 전치(transpose)하여 원래의 포맷으로 복원합니다. 
minmax_scale = preprocessing.MinMaxScaler(feature_range=(-1, 1)).fit(x.T)
x_scaled=minmax_scale.transform(x.T).T

In [ ]:
x_scaled[0]

In [ ]:
file_name='train.csv'

# 수정된 컬럼값을 데이터프레임으로 생성합니다. `Ticker`컬럼을 추가하고 이후 학습작업에서 알고리즘에 입력할 수 있도록 파일로 저장합니다. 
df = pd.DataFrame(x_scaled)
df.insert(0,'Ticker',tickers)
df.to_csv(file_name,header=False,index=False)

In [ ]:
df.head()

In [ ]:
#Next, we will upload it to Amazon S3 so that we can specify the same as part of the training job in Step 2.
train_file = sagemaker_session.upload_data(file_name, bucket, common_prefix)

#### Step 2: ML 모델 학습(Training)

AWS 마켓플리이스의 3rd party 알고리즘을 SageMaker에서 사용하기 위해 구독(subscript) 절차가 필요합니다.

1. [AWS 마켓플레이스 리스트 페이지](https://aws.amazon.com/marketplace/pp/Spotad-LTD-K-Shape-Time-Series-Clustering/prodview-bjbovimwn5ajs)에서 알고리즘을 오픈합니다.
1. **Continue to subscribe** 버튼을 클릭합니다.
1. ***Subscribe to this software*** 페이지에서 EULA, 가격, 지원정책 등 내용을 확인하고 동의하시는 경우, **"Accept Offer"** 버튼을 클릭합니다. AWS에서 실행되는 워크샾에서 제공하는 노트북에서는 이미 이 과정이 진행되었을 수 있습니다.
1. **Continue to configuration** 버튼을 클릭하고 알고리즘을 사용할 (SageMaker 노트북이 실행중인) **region** 을 선택합니다. 
1. **Product Arn**을 복사합니다. 복사한 값을 다음 셀의 `algo_arn`변수에 저장합니다.



In [ ]:
algo_arn='<아래를 참고하여 사용하시는 리전에 맞는 algorithm ARN 을 입력합니다.>'

#algo_arn='arn:aws:sagemaker:us-east-1:865070037744:algorithm/k-shape-cd639040558775d27d890f1479c92d7b'

In [ ]:
# hyperparameters 부분을 살펴보십시오. (11개의 클러스터를 위해 k=11 로, 첫번째 컬럼이 ticker를 구분하고 있으므로 label-size=1 로 지정하였습니다.)
# 인스턴스 타입을 확인하고 ML 모델 학습을 실행합니다. 
algo = AlgorithmEstimator(algorithm_arn=algo_arn, 
                          role=role, 
                          train_instance_count=1, 
                          train_instance_type='ml.m5.4xlarge', 
                          sagemaker_session=sagemaker_session, 
                          base_job_name=common_prefix,
                          hyperparameters={"k": "11", "label_size": "1"}) 

algo.fit({'train': train_file}) 


이제 알고리즘이 생성한 ML 모델을 다운로드할 수 있으며, 모델은 중심(centroid)에 대한 정보를 포함하고 있습니다. 클러스터 중심(Cluster centroid)은 각 클러스터에서 변량들의 평균입니다. 본 사례에서는 클러스터에서 발견된 시계열 관측에 대한 클러스터 중심을 시계열로 생성되었습니다. 

클러스터가 어떤 점에 소속되는지를 알기 위하여 알고리즘은 모든 클러스터의 중앙(centor)로부터 시계열들의 거리를 찾습니다. 알고리즘은 다시 가장 가까운 거리를 가지는 클러스터를 소속 클러스터로 선택할 것입니다. 

모델을 다운로드하고 클러스터 중심을 그래프로 그려보겠습니다.

In [ ]:
s3 = boto3.resource('s3')

try:
    s3.Bucket(bucket).download_file('{}/output/model.tar.gz'.format(algo._current_job_name), 'model.tar.gz')
except botocore.exceptions.ClientError as e:
    if e.response['Error']['Code'] == "404":
        print("The object does not exist.")
    else:
        raise

In [ ]:
!mkdir -p model
!tar -zxvf model.tar.gz -C model

In [ ]:
split_by_comma = lambda s: str.split(s, ',')

centroids = list()
with open('model/centroids', 'r') as f:
    for index,record in enumerate(map(split_by_comma, list(map(str.strip, f)))):
        centroid=np.array(record).astype(float)
        centroids.append(centroid)
len(centroids)

식별된 클러스터들의 중심을 그래프로 그려봅니다.


중심값들은 Z 정규화되어 있으므로 실제 주가의 범위와 매치되지는 않습니다.

In [ ]:
figure(num=None, figsize=(14, 6), dpi=150, facecolor='w', edgecolor='k')

#Display only month and day
formatter = mdates.DateFormatter("%m-%d")
ax = plt.gca()
ax.xaxis.set_major_formatter(formatter)


for index,centroid in enumerate(centroids):
    plt.plot( dates, centroid, linewidth=1, label='Centroid '+str(index))
plt.legend()


추론을 위해 ML 모델을 SageMaker 엔드포인트로 배포합니다.

In [ ]:
%%time
predictor = algo.deploy(1, 'ml.m5.4xlarge', serializer=csv_serializer)

In [ ]:
single_result=df.head(1).values[0]
result=predictor.predict(np.array(single_result[1:])).decode('utf-8')
result

<font color='red'> 워크샵에 참여하시는 분들의 과제 : 전체 학습데이터셋에 대하여 추론을 실행하고 각 행(row)에 대한 클러스터 아이디를 식별하십시오. 각 클러스트들을 별도의 선그래프로 그려서 가시화하십시오. </font>  
<font color='red'> 그리고, 여러분이 발견한 내용을 정리하십시오. </font>

In [ ]:
#predictor.delete_endpoint()

축하합니다. K-shape기반의 시계열 클러스터링을 성공적으로 수행하셨습니다. 

#### 과제 요약

<font color='red'>Task: 다음으로 팀내 구성원들끼리 나누어서 K값을 2부터 20까지 변화시키면서 실험을 진행해 보십시오. (Step 2 모델 학습 단계) 그리고 다음 섹션에서 SSD(Sum Square Distance)를 기록하고 비교하십시오.  </font>
    
본 실험에서는 Automatic model tuning을 실행하지 않기를 권장합니다. 본 과제의 목적은 모든 팀 멤버가 문제 해결을 위한 실험과정을 이해한 후 두번째 Task 2를 보다 효과적으로 해결하는 것입니다. (Task 2를 완료하고 시간이 남는 경우 추가로 진행해도 좋겠습니다.)

실험을 위해 다른 종목과 날짜 범위를 선택할 수 있습니다. 이 경우, 다음 범위내에서 지정하도록 가이드드립니다.:
1. 50개 종목 이상 
2. 3개월 이상의 기간



##### 실험결과 정리 샘플:
<font color='red'>Tickers =[]

Date range=[]


| K      | Sum Square Distance |
| ----------- | ----------- |
| ...      | ...       |
| ...      | ...       |



다음 질문에 대답해 봅니다. 
* "K"값이 얼마일 경우 가장 좋은 결과를 보이나요?
* 동일 섹터의 종목이 모두 유사한 모양을 가지고 있나요?
* 그래프를 통해 어떤 흥미로은 패턴을 발견할 수 있나요?
    
</font>

팀원이 각각 과제를 끝내고 나면, Task2는 팀전체가 함께 작업합니다. 


### Task 2 설명

주식시장의 변덕스러운 특징에도 불구하고, 많은 사람들이 주식에 투자합니다. 투자자들은 고유한 주식 포트폴리오를 고민합니다. 포트폴리오를 구성하는 수량과 구매 날짜는 다양하겠지만, 투자자들의 포트폴리오가 유일할 가능성은 매우 낮을 것입니다.  

본 과제에서는 10만명의 투자자들에 대하여 3개~10개의 종목으로 구성된 가상의 포트폴리오를 생성하고 클러스터링해 보겠습니다.


##### Step 1:

주어진 종목 집합(SPDR SP 500 ETF:(SPY)의 서브그룹) 에 대한 투자자 2만명의 포트폴리오를 생성합니다. 각 투자자에게는 고유의 아이디를 부여합니다. 


##### Step 2:
       
K-Means 클러스터링 알고리즘을 이용하여 포트폴리오에 대한 클러스터링을 수행합니다. 

    K-means는 비지도 학습 알고리즘입니다. 이 알고리즘은 가능한한 서로 유사한 멤버 데이터를 가지고, 가능한한 서로 다른 클러스터를를 가지게 되는 고유의 그룹을 찾습니다. 알고리즘 실행과정에서 유사도를 결정할 속성들을 정의할 것입니다. Amazon SageMaker는 수정된 버전의 web-scale k-means 클러스터링 알고리즘을 제공합니다. 수정된 알고리즘은 원본에 비하여 조금 더 높은 정확도를 제공하며, 원본 알고리즘과 같이 대량 데치터셋에 대한 확정성을 제공하고, 보다 빠르게 학습할 수 있습니다. 
    
K-means에 대한 보다 상세한 내용은 [개발자 가이드](https://docs.aws.amazon.com/sagemaker/latest/dg/k-means.html)를 참고합니다.
##### Step 3:
투자자 아이디를 입력받고 해당 투자자와 유사한 포트폴리오를 가지는 다른 투자자를 식별하는 함수를 생성합니다. 

서로 다른 "K"값을 이용하여 실험을 진행하고 결과를 정리합니다. 

**구현 팁**: 지연시간을 최소화하기 위해 작은 데이터셋에서부터 시작하고 큰 데이터셋에 대한 실험으로 확장해 갑니다.

### Step 1: 포트폴리오 생성


`데이터프레임 컬럼 가이드`: ['TRADER_ID','Ticker1','Ticker2','Ticker3','Ticker4'...'TickerN']

In [ ]:
num_traders=20000
min_stocks_in_portfolio=3
max_stocks_in_portfolio=10

In [ ]:
df = pd.DataFrame(columns=tickers)

In [ ]:
from random import randrange

#Each trader's portfolio must contain 3 companies and at-most 10 companies.
portfolios=[]
for trader_number in range(num_traders):
    total_stocks= randrange(min_stocks_in_portfolio, max_stocks_in_portfolio)
    for num_stock in range(total_stocks):
        df.loc[trader_number, tickers[randrange(len(tickers))]] = 1

**구현 팁**: 학습에 많은 시간이 걸린다면 더 높은 사양의 인프라로 시도해 보십시오. 


### Step 2:
<font color='red'>
본 단계에서 생성된 서로다른 포트폴리오 데이터들을 클러스터링하는 ML 모델에 대한 학습코드를 작성합니다. </font>


### Step 3:
<font color='red'>
이 단계에서 투자자를 선택하는 데 필요한 코드를 작성한 다음 선택한 트레이더의 포트폴리오와 동일한 포트폴리오에 종목이 있는 다른 사람을 찾습니다. </ font>


### Step 4: Identify optimal value for K and report metrics (Optional)

다음 블로그를 참조합니다.  
https://aws.amazon.com/blogs/machine-learning/k-means-clustering-with-amazon-sagemaker/ 
    
블로그 게시물과 동일한 실험을 수행해 보고, 그래프를 플로팅하고, 결과를 정리합니다.